In [ ]:
import requests
import pandas as pd

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests


from tqdm import tqdm

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

In [ ]:
import re

In [ ]:


def sacar_actores(archivo_csv):
    pelis = pd.read_csv(archivo_csv)
    lista_pelis = pelis
    
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get("https://www.imdb.com/")

    lista_tupla_actores = []
    sleep(3)
    for tupla in tqdm(lista_pelis["ID"]):
        driver.find_element("css selector", "#suggestion-search").send_keys(tupla, Keys.ENTER)
        sleep(8)
        driver.find_element("css selector", "#suggestion-search-button").click()
        sleep(8)
        driver.find_element("css selector", "#suggestion-search").send_keys(tupla, Keys.ENTER)
   
        lista_actores = []
        for i in range(2, 12):
            try:
                lista_actores.append(driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.gWwKlt > a').text)
            except:
                break
        lista_actores.append(tupla)
        
        lista_tupla_actores.append(tuple(lista_actores))
        driver.back()
    
    df_lista_tupla_actores = pd.DataFrame(lista_tupla_actores)
    df_lista_tupla_actores.to_csv(lista_tupla_actores+ ".csv")

    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get("https://www.imdb.com/")
    lista_detalles_actores = []
    patron = '^tt'
    
    for indice, actor in tqdm(enumerate(lista_tupla_actores)):
        lista_actores = []
        for valor in lista_tupla_actores[indice]:
            list_regex = re.findall(patron, valor)   
            if len(list_regex) == 1:
                break
    
            sleep(5)
            driver.find_element("css selector", "#suggestion-search").send_keys(valor, Keys.ENTER)
            
            sleep(5)
            try:
                driver.find_element('css selector','#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-872d7ac7-0.fqEQWL > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.iUyFfD > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a').click()
                sleep(5)
            except:
                pass
            try: 
                año = driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.dEqUUl > div.sc-e226b0e3-6.CUzkx > div.sc-e226b0e3-11.kkLqLt > section > aside > div > span:nth-child(2)').text.split(":")[-1]  
                year = año[-4:]
            except:
                year = "desconocido"
            try:
                conocido = driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.ipc-primary-image-list-card__content').text
            except:
                conocido = "desconocido"
            try:
                que_hace = driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div > ul').text
            except:
                que_hace = "desconocido"
            try:
                premio_oscar = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link").text
            except:
                premio_oscar = " "
        
            lista_caracteristicas = []
            
            lista_caracteristicas.append(valor)
            lista_caracteristicas.append(year)
            lista_caracteristicas.append(conocido)
            lista_caracteristicas.append(que_hace)
            if 'Óscar' in premio_oscar:
                lista_caracteristicas.append(premio_oscar)
            else:
                lista_caracteristicas.append("No tiene Óscar")
            
            lista_detalles_actores.append(tuple(lista_caracteristicas))
            
            driver.back()

        driver.close()
    
    pd.DataFrame(lista_detalles_actores)
    df_lista_detalles_actores = pd.DataFrame(lista_detalles_actores, columns=["Nombre", "Fecha nacimiento", "Conocido por", "En que trabaja", "Premios"])
    df_lista_detalles_actores.to_csv(archivo_csv +".csv")


